# Auto-optimization - How It *should* work:

- Script generates first MCNP input based on starting user parameters:
    - Materials
    - Required temperature of interface between heat shield and blanket
    - \# of F4 tallies in blanket
        - Fatal error if thickness of any tally is a certain thickness or greater

- File runs on ROGUE

- Output file returns, script searches output for:
    - Neutron flux in each blanket cell, *B_F4_\<n\>*
    - Neutron flux to environment, *E_F4*
    - Energy deposition in each blanket cell, *B_F6_\<n\>*
    - Energy deposition in heat shield, *HS_F6*
    - Volume of each blanket cell, *B_V_\<n\>*
    - Volume of heat shield, *HS_V*
    
- From these values, calculations are made:
    - $\sigma$, the relevant cross section for blanket material, *B_SIG*. Requires = *B_F6_\<n\>*
    - ${\tau}(t)$, the rate of tritium production in each blanket cell. Requires = *B_F4_\<n\>*, *B_SIG*, *B_V_\<n\>*,
    - $T_B$, the temperature of first blanket cell, *B_T*. Requires = *B_F6_1*
    
- What gets optimized?
    - First, blanket thickness
    - Second, the outer shield
    - OPTIONAL: heat shield thickness
    - IF "*B_F4_\<last\>* " LESS THAN "*BLANKET_LOWER_FLUX_BOUND* ":
        - Decrease blanket thickness
    - IF "*E_F4* " GREATER THAN ZERO:
        - Increase outer shield thickness
    - IF $T_B$ GREATER THAN USER-SPECIFIED BLANKET TEMP:
        - Increase heat shield thickness

In [167]:
import matplotlib.pyplot as plt
import numpy as np
from mcparse import *

In [40]:
class Material:
    
    def __init__(self, name, mass_density, atomic_weight, mat_card_num, 
                 Cv=None, LiFrac=None, Li6_E=4.85):
        
        self.name = name
        self.mass_density = mass_density
        self.atomic_weight = atomic_weight
        self.Cv = Cv
        self.fLi = LiFrac
        self.enrichment = Li6_E # Li-6 enrichment
        self.numden = self.mass_density * 6.022*10**23 / self.atomic_weight
        
        master_material_cards = { "Li2TiO3" : f"m{mat_card_num}\n     "
                                            + f"3006.80c {round(LiFrac*Li6_E, 4)}\n     "     
                                            + f"3007.80c {round(LiFrac*(1-Li6_E), 4)}\n     "     
                                            +  "22048.80c 0.1666\n     "     
                                            +  "8016.80c 0.5000\n",
                                  "BeO"     : f"m{mat_card_num}\n     "
                                            +  "4009.80c 0.5\n     "
                                            +  "8016.80c 0.5\n",
                                  "Li4SiO4" : f"m{mat_card_num}       "}
        
        # Create material card
        if self.name in master_material_cards:
            self.mcard = master_material_cards[self.name]
        else:
            print(f"Unknown material: {self.name}. Create new material card.")
            
def sig(isotope, energy):
    
    '''Reads ENDF file for isotope
       and relevant cross section'''
    
    libraries = {"Li6"     : r"C:\Users\tanhe\Desktop\NSE 474\ENDF_Li6_n,alpha.txt",
                 "Isotope" : r"path\to\xsec\file"}
    
    if isotope in libraries:
        
        with open(libraries[isotope], "r") as f:
            data = f.readlines()[11:]
            
            # Separate Energy vs. Barns:
            energy_values = []
            barns  = []
            
            for line in data:
                energy_values.append(eval([x for x in data[data.index(line)].split(" ") if x][0]))
                barns.append(eval([x for x in data[data.index(line)].split(" ") if x][1]))
            
            # Calculate relevant cross section for our system
            # Assumes LINEAR interpolation!
            xsec = np.interp(energy, energy_values, barns)
            
        return xsec
    
    else:
        raise Exception("Invalid molecule.")
    
    return

def TCalc(N_iso, flux, sig, volume, endtime):
    
    '''Calculates total # of tritium
       atoms produced up to some time "t" '''
    
    t = np.linspace(0,endtime,10000)
    T3_tot = N_iso*(1 - np.exp(-flux*sig*volume*t))
    
    return T3_tot

# Material database

In [42]:
#                     NAME    MASS DENSITY,  ATOMIC WEIGHT   M CARD NUMBER  HEAT CAPACITY  Li FRAC     Li6 ENRICHMENT
Li2TiO3 = Material("Li2TiO3",     3.43,         109.76,            1,          Cv=None,  LiFrac=0.3334,   Li6_E=0.0485)
Li4SiO4 = Material("Li4SiO4",     2.221,        119.84,            2,          Cv=None,  LiFrac=0.4444,   Li6_E=0.0485)
Li2ZrO3 = Material("Li2ZrO3",     4.21,         153.1,             3,          Cv=None,  LiFrac=0.4,      Li6_E=0.0485)
FLiBe   = Material("FLiBe",       1.91,          34.95,            4,          Cv=None,  LiFrac=0.3334,   Li6_E=0.0485)

Unknown material: Li2ZrO3. Create new material card.
Unknown material: FLiBe. Create new material card.


# Input deck optimization

In [ ]:
def createInput(heatshield_material=None, blanket_material=None, outershield_material=None):
    return

    # Create new input

def readOutP(output)